# Create a table with SRA IDs, readstats QC data, and submitter metadata

**Below are the steps taken in this notebook:**
1. Installs, import Statements & Global Variable Definitions
2. Read in tables as dataframes
3. Merge dataframes
4. Write merged dataframe to data table and file


## Installs

In [1]:
%%capture
%pip install gcsfs
## capture CANNOT have comments above it
## For reading CSVs stored in Google Cloud (without downloading them first)
## May need to restart kernel after install 

In [2]:
%%capture
%pip install --upgrade --no-cache-dir --force-reinstall terra-pandas
%pip install --upgrade --no-cache-dir  --force-reinstall git+https://github.com/DataBiosphere/terra-notebook-utils
## For reading/writing data tables into pandas data frames
## May need to restart kernel after install 

## Import Statements

In [3]:
from firecloud import fiss
import pandas as pd      
import os                 
import subprocess       
import re                 
import io
import gcsfs

from typing import Any, Callable, List, Optional
from terra_notebook_utils import table, WORKSPACE_NAME, WORKSPACE_GOOGLE_PROJECT
from terra_pandas import dataframe_to_table, table_to_dataframe

## Global Variable Declarations

In [12]:
# AnVIL_HPRC WorkspaceBucket
anvil_hprc_bucket       = "gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/"

# table filenames (expected in the workspace bucket, see below)
submitter_metadata = 'HPRC_PacBio_HiFi_Metadata_Submission_WUSTL_Year2_v0.2.txt'
sra_metadata = 'metadata-11121843-processed-ok.tsv'

# submission id
submission_id = 'WUSTL_HPRC_HiFi_Year2'

# Get the Google billing project name and workspace name for current workspace
PROJECT = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE =os.path.basename(os.path.dirname(os.getcwd()))
bucket = os.environ['WORKSPACE_BUCKET'] + "/"

# Verify that we've captured the environment variables
print("Billing project: " + PROJECT)
print("Workspace: " + WORKSPACE)
print("Workspace storage bucket: " + bucket)

Billing project: human-pangenome-ucsc
Workspace: HPRC_WRANGLING_WUSTL_HPRC_HiFi_Year2
Workspace storage bucket: gs://fc-089d894b-6682-450d-8031-f17732e5ece5/


# Read In tables as dataframes

## Readstats table
Find the individual readstats output files in the table and concatenate their contents.
Add the hifi filename and sample name.

In [5]:
# readstats
readstats_df = table_to_dataframe("readstats", workspace=WORKSPACE, workspace_namespace=PROJECT)
qc_list = []

for index, row in readstats_df.iterrows():

        sample_readstats_fp = row['ReadStatsReport']
        sample_readstats_fn = os.path.basename(sample_readstats_fp)
        df = pd.read_csv(sample_readstats_fp, header=None, sep='\t')
        df = df[df[0]=='sample.fastq']
        # add sample and file name
        #df = df.append({0:'undef', 1:'sample', 2:row['sample']}, ignore_index=True)
        df = df.append({0:'undef', 1:'filename', 2:os.path.basename(row['hifi'])}, ignore_index=True)
        # remove all but the variables (total_bp, quartile_25 etc)
        df2 = df.drop(df.columns[[0, 1]], axis=1)
        # make rownames ('total_bp', 'quartile_25' etc)
        df2.index = df.iloc[:,1]
        # remove the now meaningless 'sample.fastq' filename
        df2 = df2.drop(index=['file'])
        qc_list.append(df2)

# merge
readstats_df = pd.concat(qc_list, axis=1).transpose()
readstats_df.head()

1,total_reads,total_bp,total_Gbp,min,max,mean,quartile_25,quartile_50,quartile_75,N25,N50,N75,filename
2,1930215,35517184454,35.52,85,49382,18400,16334,18171,20074,16755,18592,20554,m64043_210205_190424.hifi_reads.bam
2,1826062,33618239844,33.62,64,50543,18410,16320,18170,20096,16741,18600,20587,m64043_210207_011920.hifi_reads.bam
2,1684931,34196548457,34.20,294,60330,20295,17422,19106,22384,17754,19801,23634,m64136_210622_163607.hifi_reads.bam
2,1764223,36078913516,36.08,520,62855,20450,17467,19227,22670,17820,19987,23940,m64136_210624_033152.hifi_reads.bam
2,1253128,27082098409,27.08,345,59307,21611,18535,20226,23527,18878,20880,25135,m64043_210530_003337.hifi_reads.bam


## Submitter metadata table

In [17]:
meta_fp = os.path.join(bucket, submitter_metadata)
metadata_df = pd.read_csv(meta_fp, sep='\t')
metadata_df = metadata_df[['filename', 'sample_ID', 'instrument_model', 'shear_method', 'size_selection', 'ccs_algorithm', 
    'polymerase_version', 'seq_plate_chemistry_version', 'generator_facility', 'generator_contact']]
metadata_df.head()

,filename,sample_ID,instrument_model,shear_method,size_selection,ccs_algorithm,polymerase_version,seq_plate_chemistry_version,generator_facility,generator_contact
0,m64043_210205_190424.hifi_reads.bam,HG00423,PacBio Sequel II,Megaruptor 1,SageELF,9.0.0.92188,P2,C2,Washington University,tgraves@wustl.edu
1,m64043_210207_011920.hifi_reads.bam,HG00423,PacBio Sequel II,Megaruptor 1,SageELF,9.0.0.92188,P2,C2,Washington University,tgraves@wustl.edu
2,m64043_210209_184051.hifi_reads.bam,HG00423,PacBio Sequel II,Megaruptor 1,SageELF,9.0.0.92188,P2,C2,Washington University,tgraves@wustl.edu
3,m64043_210211_005516.hifi_reads.bam,HG00423,PacBio Sequel II,Megaruptor 1,SageELF,9.0.0.92188,P2,C2,Washington University,tgraves@wustl.edu
4,m64043_210702_183246.hifi_reads.bam,HG00544,PacBio Sequel II,Megaruptor 1,SageELF,10.1.0.115913,P2,C2,Washington University,tgraves@wustl.edu


## SRA table
Must download after finishing submission.
The downloaded table needs to be split in one row per file (instead of filename, filename2 <...> filenameN)

In [14]:
## Read in SRA file and split by filename

sra_file = os.path.join(bucket + sra_metadata)
sample_df = pd.read_csv(sra_file, sep='\t')

fnames = [x for x in sample_df.columns if x.startswith('filename')]
sra_df = pd.melt(sample_df, id_vars =['sample_name', 'accession'], value_vars = fnames, value_name='file')
sra_df= sra_df.dropna()
sra_df = sra_df[['file', 'accession', 'sample_name']]
sra_df.columns = ['filename', 'accession', 'sample']
sra_df.head()


,filename,accession,sample
0,m64043_210205_190424.hifi_reads.bam,SRR18189665,HG00423
1,m64043_210702_183246.hifi_reads.bam,SRR18189664,HG00544
2,m64136_210520_180433.hifi_reads.bam,SRR18189653,HG00544
3,m64136_210620_054327.hifi_reads.bam,SRR18189646,HG00609
4,m64043_210530_003337.hifi_reads.bam,SRR18189645,HG00738


# Merge all dataframes

In [18]:
print(sra_df.shape, readstats_df.shape, metadata_df.shape)

(89, 3) (89, 13) (89, 10)


In [19]:
merged_df = pd.merge(
    sra_df,
    readstats_df,
    on='filename')
merged_df = pd.merge(
    merged_df,
    metadata_df,
    on='filename')
merged_df.head()

,filename,accession,sample,total_reads,total_bp,total_Gbp,min,max,mean,quartile_25,...,N75,sample_ID,instrument_model,shear_method,size_selection,ccs_algorithm,polymerase_version,seq_plate_chemistry_version,generator_facility,generator_contact
0,m64043_210205_190424.hifi_reads.bam,SRR18189665,HG00423,1930215,35517184454,35.52,85,49382,18400,16334,...,20554,HG00423,PacBio Sequel II,Megaruptor 1,SageELF,9.0.0.92188,P2,C2,Washington University,tgraves@wustl.edu
1,m64043_210702_183246.hifi_reads.bam,SRR18189664,HG00544,2009252,37881441933,37.88,89,44297,18853,17346,...,20276,HG00544,PacBio Sequel II,Megaruptor 1,SageELF,10.1.0.115913,P2,C2,Washington University,tgraves@wustl.edu
2,m64136_210520_180433.hifi_reads.bam,SRR18189653,HG00544,1498910,29354505966,29.35,86,57625,19583,16825,...,22825,HG00544,PacBio Sequel II,Megaruptor 1,SageELF,10.1.0.115913,P2,C2,Washington University,tgraves@wustl.edu
3,m64136_210620_054327.hifi_reads.bam,SRR18189646,HG00609,1784291,36542655261,36.54,80,59168,20480,17480,...,24004,HG00609,PacBio Sequel II,Megaruptor 1,SageELF,10.1.0.115913,P2,C2,Washington University,tgraves@wustl.edu
4,m64043_210530_003337.hifi_reads.bam,SRR18189645,HG00738,1253128,27082098409,27.08,345,59307,21611,18535,...,25135,HG00738,PacBio Sequel II,Megaruptor 1,SageELF,10.1.0.115913,P2,C2,Washington University,tgraves@wustl.edu


# Upload To Tables

In [20]:
## Create table
#dataframe_to_table("post_sra_metadata", merged_df, WORKSPACE, PROJECT)
outf = os.path.join(bucket, submission_id + '_post_sra_metadata.tsv')
merged_df.to_csv(outf, sep="\t", index=False)